In [20]:

## !pip install flair
## !pip install sentence-transformers


In [22]:

import pandas as pd
import torch
import torch
import numpy as np
## import SentenceTransformer
from flair.data import Sentence
from flair.embeddings import WordEmbeddings, DocumentPoolEmbeddings
from flair.embeddings import DocumentRNNEmbeddings
from flair.embeddings import TransformerDocumentEmbeddings
from flair.embeddings import SentenceTransformerDocumentEmbeddings
from transformers import AutoModelForSequenceClassification, AutoTokenizer


In [6]:

similar=[
("A black dog walking beside a pool.",            "A black dog is walking along the side of a pool."),
("A blonde woman looks for medical supplies for work in a suitcase.	",
                                 " The blond woman is searching for medical supplies in a suitcase."),
  ("A doubly decker red bus driving down the road.","A red double decker bus driving down a street."),
("There is a black dog jumping into a swimming pool.","A black dog is leaping into a swimming pool."),
    ("The man used a sword to slice a plastic bottle.","A man sliced a plastic bottle with a sword.")
]


In [7]:

print(   pd.DataFrame(similar, columns=["sen1", "sen2"])    )


                                                sen1  \
0                 A black dog walking beside a pool.   
1  A blonde woman looks for medical supplies for ...   
2     A doubly decker red bus driving down the road.   
3  There is a black dog jumping into a swimming p...   
4    The man used a sword to slice a plastic bottle.   

                                                sen2  
0   A black dog is walking along the side of a pool.  
1   The blond woman is searching for medical supp...  
2     A red double decker bus driving down a street.  
3       A black dog is leaping into a swimming pool.  
4        A man sliced a plastic bottle with a sword.  


In [8]:



dissimilar= [
("A little girl and boy are reading books. ", "An older child is playing with a doll while gazing out the window."),
("Two horses standing in a field with trees in the background.", "A black and white bird on a body of water with grass in the background."),
("Two people are walking by the ocean." , "Two men in fleeces and hats looking at the camera."),
("A cat is pouncing on a trampoline.","A man is slicing a tomato."),
("A woman is riding on a horse.","A man is turning over tables in anger.")
]




In [9]:



print(    pd.DataFrame(dissimilar, columns=["sen1", "sen2"])    )




                                                sen1  \
0          A little girl and boy are reading books.    
1  Two horses standing in a field with trees in t...   
2               Two people are walking by the ocean.   
3                 A cat is pouncing on a trampoline.   
4                      A woman is riding on a horse.   

                                                sen2  
0  An older child is playing with a doll while ga...  
1  A black and white bird on a body of water with...  
2  Two men in fleeces and hats looking at the cam...  
3                         A man is slicing a tomato.  
4             A man is turning over tables in anger.  


In [10]:

def sim(s1,s2):
  # cosine similarity function outputs in the range 0-1
  s1=s1.embedding.unsqueeze(0)
  s2=s2.embedding.unsqueeze(0)
  sim = torch.cosine_similarity(s1,s2).item() 
  return np.round(sim,2)


In [11]:



def evaluate(embeddings, myPairList):
  # it evaluates embeddings for a given list of sentence pair
  scores=[]
  for s1, s2 in myPairList:
    s1,s2=Sentence(s1), Sentence(s2)
    embeddings.embed(s1)
    embeddings.embed(s2)
    score=sim(s1,s2)
    scores.append(score)
  return scores, np.round(np.mean(scores),2)




In [12]:

glove_embedding       = WordEmbeddings('glove')
glove_pool_embeddings = DocumentPoolEmbeddings([glove_embedding])


In [13]:

print("Glove just average embeddings")
print(    evaluate(glove_pool_embeddings, similar)     )
print(    evaluate(glove_pool_embeddings, dissimilar)  )


Glove just average embeddings
([0.97, 0.99, 0.97, 0.99, 0.98], 0.98)
([0.94, 0.97, 0.94, 0.92, 0.93], 0.94)


In [14]:

gru_embeddings = DocumentRNNEmbeddings([glove_embedding])


In [15]:


print("GRU RNN embeddings") 

print(    evaluate(gru_embeddings, similar)      )
print(    evaluate(gru_embeddings, dissimilar)   )




GRU RNN embeddings
([0.99, 1.0, 0.93, 1.0, 0.92], 0.97)
([0.88, 1.0, 0.91, 0.89, 0.85], 0.91)


In [16]:

bert_embeddings = TransformerDocumentEmbeddings('bert-base-uncased')


In [17]:



print(   evaluate(bert_embeddings, similar)       )
print(   evaluate(bert_embeddings, dissimilar)    )




([0.85, 0.9, 0.96, 0.91, 0.89], 0.9)
([0.93, 0.94, 0.86, 0.93, 0.92], 0.92)


In [23]:


sbert_embeddings = SentenceTransformerDocumentEmbeddings('bert-base-nli-mean-tokens')



In [24]:

print("--------------------------------")
print(  "Notice, this one actually does what we want"  )
print(   evaluate(sbert_embeddings, similar)   )
print(   evaluate(sbert_embeddings, dissimilar)   )


--------------------------------
Notice, this one actually does what we want
([0.98, 0.95, 0.96, 0.99, 0.98], 0.97)
([0.48, 0.41, 0.19, -0.05, 0.0], 0.21)
